**Initialization**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [ ]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**Downloading Libraries and Dependencies**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [22]:
#@ IMPORTING MODULES: UNCOMMENT BELOW:
# !pip install transformers[sentencepiece]
# !pip install datasets
# !pip install sacrebleu
import torch
import nltk
# nltk.download('punkt')
import pandas as pd
import numpy as np
import torch.nn.functional as F
from nltk.tokenize import sent_tokenize
from datasets import load_dataset
from datasets import load_metric
from transformers import pipeline
from transformers import set_seed
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

#@ IGNORING WARNINGS: 
import warnings
warnings.filterwarnings("ignore")

**The CNN/DailyMail Dataset**
- This dataset consists of around 300,000 pairs of news articles and their corresponding summaries, composed from the bullet points that CNN and DailyMail attach to their articles.

In [ ]:
#@ LOADING THE DATASET:
dataset = load_dataset("cnn_dailymail", version="3.0.0")               # Loading the dataset.
print(f"Features: {dataset['train'].column_names}")                    # Inspecting columns.

Using custom data configuration default
Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

Features: ['article', 'highlights', 'id']


In [ ]:
#@ INSPECTING SAMPLE ARTICLES:
sample = dataset["train"][1]                                    # Initialization.
print(f"Article (excerpt of 500 characters, total length:\
{len(sample['article'])})")                                     # Inspecting number of characters.
print(sample["article"][:500])                                  # Inspecting article.
print(f"\nSummary length: {len(sample['highlights'])}")         # Inspecting length of summary.
print(sample["highlights"])                                     # Inspecting summary.

Article (excerpt of 500 characters, total length:3192)
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has n

Summary length: 180
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .


**Text Summarization Pipelines**

In [ ]:
#@ INITIALIZING TEXT SUMMARIZATION:
sample_text = dataset["train"][1]["article"][:2000]             # Initialization.
summaries = {}                                                  # Initializing a dictionary.

#@ SENTENCE TOKENIZATON: EXAMPLE:
string = "The U.S. are a country. The U.N. is an organization." # Text example.
sent_tokenize(string)                                           # Inspection.

['The U.S. are a country.', 'The U.N. is an organization.']

In [ ]:
#@ INITIALIZING SUMMARIZATION BASELINE:
def three_sentence_summary(text):                               # Defining function.
    return "\n".join(sent_tokenize(text)[:3])                   # Getting 3 sentences.
summaries["baseline"] = three_sentence_summary(sample_text)     # Getting baseline summary.

**GPT-2**

In [ ]:
#@ INITIALIZING SUMMARIZATION WITH GPT-2:
set_seed(2022)
pipe = pipeline("text-generation", model="gpt2")                # Initializing pipeline.
gpt2_query = sample_text + "\nTL;DR:\n"                         # Initialization.
pipe_out = pipe(gpt2_query, max_length=512, 
                clean_up_tokenization_spaces=True)              # Generating summaries.
summaries["gpt2"] = "\n".join(sent_tokenize(
    pipe_out[0]["generated_text"][len(gpt2_query):]
))                                                              # Adding to summaries.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


**T5**

In [ ]:
#@ INITIALIZING SUMMARIZATION WITH T5:
pipe = pipeline("summarization", model="t5-small")              # Initializing T5 pipeline.
pipe_out = pipe(sample_text)                                    # Generating summaries.
summaries["t5"] = "\n".join(sent_tokenize(
    pipe_out[0]["summary_text"]
))                                                              # Adding to summaries.

**BART**

In [14]:
#@ IMPLEMENTATION OF BART:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")   # Initializing BART pipeline.
pipe_out = pipe(sample_text)                                        # Generating summaries.
summaries["bart"] = "\n".join(sent_tokenize(
    pipe_out[0]["summary_text"]
))                                                                  # Adding to summaries.

**PEGASUS**

In [16]:
#@ INITIALIZING SUMMARIZATION WITH PEGASUS:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail") # Initializing T5 pipeline.
pipe_out = pipe(sample_text)                                           # Generating summaries.
summaries["t5"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")  # Adding to summaries.

**Comparing Different Summaries**

In [17]:
#@ COMPARING DIFFERENT MODEL SUMMARIES: 
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])                 # Initializing baseline summaries.
print(" ")

for model_name in summaries:
    print(model_name.upper())                            # Initializing model name.
    print(summaries[model_name])                         # Getting summaries. 
    print(" ")

GROUND TRUTH
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .
 
BASELINE
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.
The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.
The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover.
 
GPT2
United States Team
 
T5
Usain Bolt wins third gold of world championships.
Anchors Jamaica to victory in men's 4x100m relay.
Eighth gold at the championships for Bolt.
Jamaica also win women's 4x100m relay .
 
BART
Usain Bolt wins his third gold of the world championships in Moscow.
Bolt anchors Jamaica to victor

**Quality Measurement**

In [24]:
#@ INITIALIZING QUALITY MEASUREMENT: EXAMPLE: 
bleu_metric = load_metric("sacrebleu")                                  # Initializing BLEU.
bleu_metric.add(prediction="the the the the the the",
                reference=["the cat is on the mat"])                    # Initializing example.
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)    # Computing bleu score. 
results["precisions"] = [np.round(p, 2) for p in results["precisions"]] # Adding arrays of precisions.
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])      # Creating a dataframe.

,Value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


In [25]:
#@ INITIALIZING QUALITY MEASUREMENT: EXAMPLE: 
bleu_metric = load_metric("sacrebleu")                                  # Initializing BLEU.
bleu_metric.add(prediction="the cat is on mat",
                reference=["the cat is on the mat"])                    # Initializing example.
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)    # Computing bleu score. 
results["precisions"] = [np.round(p, 2) for p in results["precisions"]] # Adding arrays of precisions.
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])      # Creating a dataframe.

,Value
score,57.893007
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


**ROUGE**